# Job Title Classifier
by Santiago M.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import nltk
import torch
import pandas as pd

import numpy as np
from sklearn.cluster import KMeans

!pip install nmslib 
import nmslib

nltk.download('punkt')



     |████████████████████████████████| 13.0MB 2.8MB/s 
     |████████████████████████████████| 153kB 57.8MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
! mkdir NLP
%cd NLP

# !mkdir GloVe
# !curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip GloVe/glove.840B.300d.zip -d GloVe/

!mkdir fastText
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

/content/NLP
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  23.0M      0  0:01:03  0:01:03 --:--:-- 23.4M
Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [4]:
!mkdir encoder
# !curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  20.1M      0  0:00:07  0:00:07 --:--:-- 23.8M


In [5]:
! git clone https://github.com/facebookresearch/InferSent.git

from InferSent.models import InferSent


Cloning into 'InferSent'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 259 (delta 7), reused 13 (delta 4), pack-reused 240
Receiving objects: 100% (259/259), 448.95 KiB | 11.22 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [0]:
sentences = [
    "Executive Director for Tax",
    "ASSOCIATE PUBLIC HEALTH SANITARIAN II",
    "Investigator",
    "MAINFRAME STORAGE MANAGER",
    "Civil Engineer 2",
    "2020-WT-008-Asset Management Intern",
    "VP- Operations",
    "IT Project Manager/Business Analyst",
    "MAINTENANCE WORKER",
    "Project Advocate",
    "Labor Compliance Analyst",
    "Forestry Program Manager",
    "JAVA DEVELOPER",
    "DEPUTY DIRECTOR -  Strategic Initiatives &Analysis",
    "City Seasonal Aide",
    "EXECUTIVE DIRECTOR, REGULATORY REFORM",
    "Deputy Director Conversions/HQE",
    "Administrative Assistant",
    "Watershed Maintainer",
    "Supervisor Const. & Performance Monitoring",
    "Executive Project Audit Engineer",
    "Community Associate",
    "Federal Transportation & Environmental Policy Advisor",
    "Construction Project Manager",
    "AGENCY COUNSEL",
    "AGENCY ATTORNEY 02",
    "Facilities & Security Coordinator",
    "ACCOUNTABLE MANAGER",
    "Payment Analyst",
    "Quality Assurance Engineer",
    "PROGRAM MANAGER",
    "Project Manager Intern",
    "Collections Unit Head - OCA/External Inquiry",
    "Supervisor/Auditors of Accountability and Audit Unit",
    "Confidential Investigator",
    "DOT Scope Development Coordinator",
    "Supervising Health Nurse, Bureau of School Health/SH Nursing Services & Prof Dev",
    "Borough Planner",
    "Deputy Inspector General",
    "Systems Engineer",
    "Senior Enforcement Counsel",
    "Recoupment Specialist",
    "URBAN DESIGNER - STATEN ISLAND BOROUGH OFFICE",
    "Executive Assistant",
    "Cosntruction Project Manager",
    "Agency Implementation and Support Program Evaluator",
    "2020-EHS-001-Performance Measurement Intern",
    "Public Health Nurse, Bureau of School Health",
    "Summer Graduate Intern",
    "DIRECTOR, WORKFORCE1 CANDIDATE SERVICES",
    "Assistant Corporation Counsel",
    "Product Manager, MO for Economic Opportunity",
    "Forensic Pathology Technician",
    "Senior Port Engineer",
    "Community Coordinator",
    "Solutions Architect, Enterprise Data Services",
    "Education Analyst",
    "BUDGET MANAGER FOR MICSA/ADULT PROTECTIVE SERVICES",
    "Assistant District Attorney - Special Victims Bureau",
    "Special Advisor: Public, Private Partnerships",
    "Web Application Developer",
    "Project Manager",
    "JUNIOR OPERATOR",
    "IT and Security Auditor (3 positions)",
    "Cyber Security Intern Technology Lab",
    "Associate Chemist 1",
    "RESIDENT BUILDING SUPERINTENDENT (HA)",
    "A-Team Supervisor",
    "Civil Engineer/Project Manager",
    "ASSISTANT CORPORATION COUNSEL",
    "Electrical Engineer 1",
    "Alternative to Placement (ATP) Assessor",
    "Application Support Reporting Specialist",
    "ASSISTANT CIVIL ENGINEER",
    "Computer Associate",
    "Design Reviewer (Structural)",
    "QUALITY ASSURANCE SPECIALIST",
    "Agency Attorney, Level I",
    "Deputy Commissioner, Wastewater Treatment",
    "Receptionist",
    "Senior Project Manager",
    "Water Use Inspector",
    "Agency Attorney III - Strategic Initiatives & Special Policy / Public Information",
    "Clerical Associate 3",
    "Machinist's Helper",
    "Senior Investment Officer - Infrastructure",
    "BIM/CAD Administrator",
    "Supervising Analyst / Unit Head  Accounting Services",
    "Record Management Officer",
    "Deputy Director",
    "Analyst, Project Management Office",
    "SENIOR SPECIAL INVESTIGATOR",
    "Accountable Manager, Resiliency Coordination and I&I Initiatives",
    "NYC Census 2020 - Senior Advisor, Communications",
    "Assistant Landscape Architect",
    "Deputy Budget Director",
    "Highway Transportation Specialist",
    "Data Scientist",
    "2020-BPAC-002-Water-On-The-Go Intern",
    "Labor Relations Intern",
    "AUTO MECHANIC (4 Day Temp)",
    "Management Auditor",
    "Analyst, Finance",
    "MACHINIST'S HELPER",
    "Senior Program Manager",
    "CERTIFIED IT ADMINISTRATOR (LAN/WAN)",
    "ELIGIBILITY WORKER",
    "WEB APPLICATION DEVELOPERS",
    "Section Chief, Green Infrastructure Maintenance",
    "Agency Attorney III",
    "Program Manager",
    "Forensic Engineering Intern",
    "Deputy Inspector General for Audit",
    "City Park Worker",
    "PROCUREMENT/NEGOTIATIONS ATTORNEY",
    "Senior Researcher",
    "COMPUTER SPECIALIST (SOFTWARE)",
    "Program Officer - I",
    "PLASTERER (HOUSING AUTH-40 HRS",
    "Youth Development Specialist",
    "HR Generalist/Civil Service Specialist",
    "Assistant Civil Engineer",
    "PLASTERER",
    "2020-BWS-006-WRRO Intern (Grahamsville)",
    "Data Analyst, PNA",
    "Director, Business Enterprise Systems",
    "Program Associate",
    "Assistant Director  Citywide Savings",
    "Senior Police Administrative Aide",
    "Domestic Violence Liaison",
    "COMPLIANCE AUDITOR",
    "CHILD ASSESSMENT SPECIALIST (BRONX)",
    "Data Analyst",
    "DIRECTOR, WOMEN ENTREPRENEURS NYC (WE NYC)",
    "Plan Examination Intern",
    "ARCHITECTURAL INTERN",
    "CERTIFIED IT ADMINISTRATOR (WAN)",
    "ADVISOR",
    "Deputy Press Secretary",
    "Staff Analyst",
    "Field Responder, Bureau of Alcohol and Drug Use, Prevention, Care, and Treatment",
    "Administrative Budget Supervisor",
    "Senior Counsel",
    "SUPERVISOR OF MECHANICAL INSTALLATIONS & MAINTENANCE",
    "AUTO MECHANIC",
    "Electrical Engineering Intern",
    "Air Compliance Specialist",
    "2020-WT-003-Mechanical Contracts",
    "Investment Officer - Public Equity",
    "Certified IT Developer (Applications) Level 3",
    "Director of Financial Equity & Asset Building",
    "INSTRUMENTATION SPECIALIST I",
    "BUYER",
    "Lead Abatement Worker",
    "Vulnerability Specialist",
    "CITY ATTENDANT",
    "CHIEF OF SHAFT MAINTENANCE",
    "Ombudsperson/Pro Se Clerk",
    "COMPUTER SYSTEMS MANAGER",
    "Environmental Planning Specialist",
    "Analyst  Technology Budget & Management",
    "ENROLLMENT SPECIALIST",
    "IT and Security Auditor",
    "Principal Administrative Associate",
    "Public Health Communications Coordinator",
    "Compliance Analyst",
    "Assistant Commissioner, Capital Planning & Delivery",
    "Laboratory Microbiologist, Bureau of the Public Health Laboratory",
    "2020-EEO-001-EEO Intern",
    "Design Reviewer, New Construction",
    "Head of Short Term Strategic Portfolio and Trading",
    "Junior Linux Full Stack Developer",
    "Contract Manager",
    "Industrial Control Technician",
    "WEB APPLICATION DEVELOPER",
    "Analyst, Load Management",
    "Bureau Staff Attorney",
    "Senior Software Developer",
    "2020-ODHR-001-Workforce Development & Training Intern",
    "E-Learning Content Developer",
    "Watershed Maintenance",
    "Stationary Engineer (Electric)",
    "Audit Supervisor (Engineering Audits)",
    "Associate - Community Action Center (CAC)",
    "2020-BWS-009-EHS Core Services Intern",
    "ELECTRONIC APPLICATION COUNCELOR (EAC)",
    "City Custodial Assistant, Bureau of Operations",
    "Senior Design Reviewer (Structural)",
    "CMS Developer",
    "Programs Data Manager",
    "Elevator Inspector",
    "2020-BWS-001-DSP Water Quality Intern",
    "Victim Advocate",
    "Gardener",
    "Bureau Energy Manager",
    "Agency Attorney I",
    "Clerical Associate Level IV",
    "Engineering Technician I",
    "Reviewer at Cross Connection Control Unit",
    "Agency Attorney 2",
    "Chief, Contract Support Unit",
    "Supervising Physician, Bureau of School Health",
    "Assistant General Counsel",
    "Project Executive",
    "Timekeeper",
    "Assistant Environmental Engineer",
    "Watershed Maintainer â 3 positions",
    "Forensic Coordinator",
    "Senior Policy Analyst, Performance Reporting Team",
    "Computer Specialist (Software)",
    "Tenant Advocate Intern",
    "TEMPORARY POPS PROGRAM SPECIALIST",
    "Clerical Associate III",
    "Assistant Commissioner, Data Analytics",
    "GIS Developer",
    "EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",
    "AGENCY ATTORNEY",
    "Investigative Attorney",
    "Applications Worker",
    "Civil Litigation General Litigation Agency Attorney Level III",
    "Engineer-In-Charge, Engineering and Metrics Unit",
    "PAYROLL EXAMINER",
    "Clerical Associate 4",
    "Office Clerk AOT, Bureau of Mental Health",
    "GIS Analyst",
    "Junior Telecom Support Technician",
    "Technical Project Manager",
    "Facilities Building Custodian",
    "FMS Acceptance Coordinator",
    "Network Engineer, Solutions and Logistics",
    "Bookkeeper, Level III",
    "Supervisor (Watershed Maintenance)",
    "Bike Share Program Coordinator",
    "Sewage Treatment Worker",
    "EHS Management Systems Specialist",
    "IT Systems Support Engineer",
    "Associate Investigator",
    "Cost Estimating Manager",
    "Supervisory Control and Data Acquisition (SCADA) Manager",
    "Associate Project Manager 02",
    "ELECTRICAL ENGINEERING INTERN",
    "Unit Chief, Interoperability Unit",
    "Supervisor Dockbuilder",
    "Coordinator: Healthy Aging Initiatives",
    "STRUCTURAL ENGINEER TRAINEE",
    "Analysis and Reporting Analyst",
    "Emergency Manager",
    "Secretary IV",
    "2020-WT-004-Mechanical Engineer",
    "Scanning Operator, Bureau of Vital Statistics",
    "Safety Standards and Guidance Intern",
    "Environmental Specialist",
    "Labor Compliance Officer",
    "Temporary Painter",
    "Housing Inspector",
    "Policy Analyst for Accessibility",
    "Contract Administrator",
    "Associate Zoning Planner",
    "2020-BWS-002-Grahamsville Laboratory Intern",
    "PROCUREMENT SPECIALIST",
    "Bid Package Reviewer",
    "Integrity Review Specialist",
    "Health and Safety Specialist",
    "Administrative Staff Analyst (Non-Managerial)",
    "RES Energy Coordinator",
    "Junior .NET Developer",
    "2020-PS-002-Emergency Response & Technical Assessment",
    "Financial Auditor",
    "2020-WT-006-Project Development",
    "PROGRAM MANAGER, REGULATORY REFORM",
    "Senior Capital Budget Analyst",
    "Child Care Inspector, Bureau of Childcare",
    "Civilian Investigator",
    "Licensing Specialist",
    "Field Inspector",
    "Senior Windows Administrator",
    "Senior Sustainability Coordinator",
    "Executive Director of Credit & Special Underwriting, Office of Policy & Strategy",
    "Driver Assistance Center Program Manager",
    "Business Analyst",
    "Staff Analyst Level II",
    "2020-SUST-002-Green Infrastructure Intern",
    "SENIOR PEOPLESOFT HCM ANALYST",
    "Data Analytics Specialist",
    "Networking Intern",
    "Deputy Commissioner, Family & Child Health",
    "Assistant Counsel",
    "PREVAILING WAGE INVESTIGATOR",
    "Hiring Plan Analyst",
    "Supervising Public Health Nurse, Bureau of School Health/SH Nursing Services & Prof Dev",
    "Tracking and Monitoring Data Analyst",
    "Plumbing Inspector",
    "College Aide L1",
    "Information Technology Services Summer Intern",
    "Chief Officer Safety and Tactics Advisor",
    "Field Technician",
    "LEAD DESIGNER FOR WATER TUNNEL STRUCTURAL",
    "Agency Counsel",
    "Child Protective Manager",
    "Thermostat Repairer",
    "HR Intern, HR Unit",
    "OFFICE LIAISON",
    "Plumbing Engineer",
    "ELECTRICIAN",
    "Dam Safety Coordinator",
    "2020-BWS-005-Water Quality Lab Intern (Kingston)",
    "Maintenance Worker - Technical Services-Heating Unit",
    "Assistant Commissioner, Bureau of Mental Hygiene Community Engagement, Policy and Practice",
    "BOROUGH PLANNER",
    "Information Representative",
    "Legal Assistant",
    "Project Administrator",
    "Network & Systems Engineer",
    "ASSISTANT COMMISSIONER, PUBLIC AFFAIRS",
    "Deputy Director of Payroll and Timekeeping",
    "Administrative Contract Specialist M-I",
    "Principal Administrative Associate - Level I",
    "2020-BEDC-003-WSCP Project Support Intern",
    "DIRECTOR, DATA AND OPERATIONS",
    "CAD and Records Manager",
    "ELECTRICAL ENGINEER",
    "Case Analyst",
    "Employer Engagement Coordinator",
    "Civil Engineer I",
    "Administrative Budget Analyst",
    "MECHANICAL ENGINEERING INTERN",
    "Administrative Associate",
    "Senior Program Officer, Asset Building",
    "E-Learning Content Developer (Per Diem)",
    "Connections and Title IV-E liaison",
    "2020-WT-001-Electrical & Instrumentation Contracts",
    "2020-BPAC-001 Environmental Educational Intern",
    "Facilities Assistant",
    "Capacity Building Specialist",
    "DIRECTOR OF DATA ANALYTICS AND RESEARCH",
    "Pest Control Aide, Bureau of Veterinary and Pest Control Services",
    "IMAGING SUPPORT CLERK",
    "PROGRAM AND POLICY  Unit Head",
    "Engineer-In-Charge",
    "Senior Project Controls Manager",
    "Senior IT Developer",
    "CONTRACTS ATTORNEY",
    "MACHINIST",
    "Assistant District Attorney - Criminal Court",
    "Mechanical Engineer",
    "Landscape Architect",
    "Chief Information Officer",
    "Project Advocate Intern",
    "STRATEGIC PROJECTS MANAGER",
    "Deputy Commissioner, Water and Sewer Operations",
    "Juvenile Justice Investigator Supervisor",
    "Hearing Officer",
    "STRUCTURAL ENGINEER",
    "Performance Analyst",
    "PROGRAM MANAGER, CANDIDATE SERVICES",
    "Machinist",
    "Information Technology Program Director",
    "DIRECTOR, PROGRAM MANAGEMENT OFFICE",
    "PH Adviser Optometry Eyeglass",
    "Chief of Staff to the Assistant Commissioner of Human Resources",
    "Director of Administration",
    "Heating Oversight Team Specialist",
    "EARD TEAM LEADER",
    "Pipeline Planner",
    "Energy Manager",
    "Division Chief Contracting and Budgeting",
    "SUMMER GRADUATE INTERN",
    "Deputy Director of Field Operations",
    "SENIOR DEVELOPER",
    "DEPUTY DIRECTOR, PARTNERSHIPS AND COMMUNICATIONS",
    "MEDICOLEGAL ANALYST",
    "WATERSHED MAINTAINER",
    "Quality Assurance Manager, Crime Scene Unit, M-II",
    "Cyber Security Engineer",
    "F/T Computer Aide",
    "Program Associate, Clean Energy",
    "Client Engagement Advisor, Workforce Development",
    "Resiliency Engineer",
    "NYC Census 2020 - Field Director",
    "Fiscal Analyst",
    "ACRIS/DTM/OCR Director",
    "DIRECTOR, BUSINESS PROGRAMS",
    "Analyst Sanitation",
    "Project Coordinator, Administration Office",
    "Executive Agency Counsel",
    "Director of Senior Practice Consultation",
    "Labor Relations Associate",
    "Forester",
    "Unit Head / Deputy Assistant Director  Capital Program Scope Development (CPSD)",
    "2020-BWS-014-Reservoir Ops Associate Data Analyst Intern",
    "2020-ODHR-002-Human Resources Information Services Intern",
    "Geotechnical Project Manager",
    "Budget Analyst",
    "Procurement Analyst II",
    "Waterfront Facilities Engineer",
    "CERTIFIED IT ADMINISTRATOR (WAN), Level 4",
    "Claim Specialist",
    "Police Administrative Aide",
    "Elevator Oversight Team (EOT) Administrator",
    ".Net Programmer / Analyst",
    "Construction Inspector",
    "Data Coordinator",
    "Supervisor of the Payroll NYCAPS Unit",
    "Senior Estimator (Mechanical)",
    "APA Foster Care Program Evaluator",
    "Investigator (Employee Discipline)",
    "Contracts Officer",
    "Nutrition Consultant",
    "Clerical Associate, Bureau of Communicable Diseases",
    "Senior Investment Officer â Real Estate",
    "Press Secretary",
    "Assistant for Economic Development",
    "Deputy Director of Analytics and Reporting",
    "City Research Scientist II",
    "Assistant Project Manager",
    "Contract Analyst",
    "Compliance Specialist",
    "City Park Worker (Full-Time)",
    "Director, Central Records Division",
    "Summons/Violation Coordinator",
    "Director of Quality Assurance and Improvement",
    "Crime Analyst, Level I",
    "Transit Benefit Clerk",
    "Call Center, Correspondence, and Lost Property Agent",
    "2020-WT-002-Electrical Engineering",
    "Fire Safety Oversight Team (FSOT) Specialist",
    "College Aid-Graphic Designer",
    "DIRECTOR, WORKFORCE1 TRAINING",
    "311 Call Center Manager",
    "Section Chief Catch Basin Inspection Unit",
    "HEARING OFFICER",
    "DEPUTY DIRECTOR OF CAPITAL BUDGET",
    "Deputy Chief, Permitting",
    "DIRECTOR OF CONSTRUCTION / SMALL PROJECTS",
    "Engineer-In-Charge, Contract Support Unit",
    "Administrative Engineer",
    "Clinical Advisor",
    "Principal Administrative Associate I",
    "Analyst  MISCELLANEOUS REVENUE",
    "APPLICATION DEVELOPER",
    "Payroll Funding Analyst",
    "Associate Public Health Sanitarian 1",
    "Bilingual NFP Nurse Home Visitor, Bureau of Maternal Infant and Reproductive Health",
    "DEPUTY COMMISSIONER",
    "Infrastructure Summer Intern",
    "EEO Investigator",
    "Cyber Audit Analyst",
    "Senior Advisor",
    "Green Infrastructure Program Analyst",
    "MOME Broadcast and IT Support Engineer",
    "GIS CSCL Coordinator",
    "Engineer-In-Charge, Survey and Investigations Unit",
    "Analyst  FIRE",
    "Prosecuting Attorney",
    "2020-BWS-013-Regulatory & Engineering Program Intern WOH",
    "Compliance Review Unit Supervisor",
    "DCAS Safety and Health Officer",
    "Environmental Health & Safety Regional Manager",
    "PEOPLESOFT ADMINISTRATOR",
    "Executive Director",
    "Deputy Director Quality Improvement, Mental Health",
    "Executive  Communications Associate",
    "Project Controls Summer Intern",
    "Technical Advisor",
    "Deputy Director, Neighborhood Planning",
    "Facility Operations Custodial Assistant",
    "Junior Compliance Investigator",
    "Associate Project Manager",
    "Program Specialist",
    "Clerical Associate- Appeals",
    "Cyber Security Intern - Cyber Audit",
    "Community Assistant, Part Time",
    "Senior Project Controls Specialist",
    "Senior Design Reviewer (Civil)",
    "Senior Policy Advisor, Climate Partnerships",
    "Computer Specialist",
    "Construction Project Manager 2",
    "Discriminatory Harassment Investigator",
    "Staff Attorney",
    "Director of Operations, NYC Connected",
    "PAYMENT ANALYST",
    "POPS PROGRAM SPECIALIST",
    "Analyst  HEALTH + HOSPITALS",
    "PROJECT MANAGER",
    "COMPUTER SYSTEMS MANAGER, M-2",
    "Forestry Supervisor",
    "On-Line Learning Instructional Design Specialist",
    "Community Coordinator (Vendor Payment)",
    "AGENCY ATTORNEY  03",
    "General Counsel / Deputy Comptroller for Legal Affairs",
    "Civil Engineer",
    "Vehicle Licensing Specialist",
    "Principal Administrative Associate - Level III",
    "Infrastructure Project Manager",
    "BOOKKEEPER",
    "Associate Staff Analyst",
    "2020-BLA-001 Undergraduate Legal Intern",
    "Mechanical Systems Inspector",
    ".NET DEVELOPER",
    "Workplace Violence Investigator",
    "ASSISTANT DIRECTOR, FINANCIAL CAPABILITY AND BUSINESS DEVELOPMENT",
    "Taxi and Limousine Inspector - Enforcement",
    "Deputy Director - Research & Investigations",
    "Investigative Intern, Internal Affairs and Discipline (IAD)",
    "Public Health Advisor, Bureau of STD Prevention and Control",
    "Landscape Architect Intern",
    "Software Developer",
    "EXECUTIVE DIRECTOR, CONSTRUCTION SAFETY TRAINING",
    "Help Desk Level 1 Representative",
    "Safety Manager",
    "Conflict Resolution Specialist",
    "Director of Upstate Operations",
    "EHS Compliance Systems Specialist",
    "CLAIM SPECIALIST Level 1",
    "Student Legal Specialist",
    "Automated Enforcement Camera Technician (AEU)",
    "Engineering Technician IV",
    "Health & Safety Specialist",
    "Government Affairs Officer/Legislative Analyst",
    "Budget Analyst, Bureau of Budget and Revenue/Budget Administration",
    "STATIONARY ENGINEER",
    "Senior Trainer",
    "Energy Liaison Coordinator",
    "CLERICAL ASSOCIATE",
    "Safety & Site Support Summer Intern",
    "CATCH School Nurse,  Bureau of School Health/SH Reproductive Health",
    "City Seasonal Aide/Security Guard",
    "Analyst  Police",
    "Deputy Director for Manhattan Planning",
    "Director of Labor Compliance",
    "Civil Engineer II",
    "Construction Project Manager for FHNR",
    "Custodian",
    "Supervisor Watershed Maintenance I",
    "Supervisor, EHS Facility Compliance â Downstate Division",
    "Assistant Director of Grants",
    "Associate Water Use Inspector",
    "Urban Designer â Bronx Borough Office",
    "Administrative Transportation Coordinator M-I",
    "ASSISTANT RESIDENT BUILDING SUPT",
    "Energy Project Manager - Capital",
    "BUSINESS ANALYST",
    "ASSISTANT PROJECT MANAGER",
    "Policy Advisor",
    "Supervising Attorney-Housing & Public Accommodations",
    "ETL Developer, MO for Economic Opportunity",
    "Environmental Health & Safety Incident Investigator",
    "Design Engineer",
    "Deputy ACCO",
    "Director, Project Management Office",
    "Engineering Intern",
    "PROJECT MANAGER, PMO",
    "ECONOMIC ANALYSIS UNIT  ANALYST",
    "Traffic Device Maintainer",
    "CERTIFIED IT DEVELOPER (APPLICATIONS)",
    "Computer Systems Manager",
    "Senior Detective Investigator",
    "Managing Director of Green Infrastructure Design and Construction",
    "CARETAKER J",
    "Supervisor (Watershed Maintenance) I",
    "Emergency Management Training & Exercise Coordinator",
    "REVENUE ANALYST",
    "Court Representative, Litigation Division (3 Positions)",
    "CoC Contract Coordinator",
    "Analyst  DOE Expense",
    "2020-SUST-001-Air Compliance Intern",
    "Mental Health Program Associate",
    "Supervising Analyst / Unit Head  Pension Analysis",
    "Community Affairs Specialist",
    "Public Buildings Summer Intern",
    "Ordered Deductions Clerk",
    "2020-FMC-001-Architect/Engineering Intern",
    "Senior Estimator (Electrical)",
    "Collections Attorney",
    "Division Chief of Construction Management",
    "Senior Budget Analyst",
    "Water Use Inspector II",
    "Pension Representative",
    "Urban Planner",
    "Stationary Engineer",
    "SENIOR PROJECT LEADER",
    "Project Manager (GIS)",
    "CONFIDENTIAL INVESTIGATOR",
    "Deputy Chief of Distribution Water Quality Field Operations",
    "Assistant Mechanical Engineer",
    "Community Service Technical Liaison",
    "Clerical Supervisor (ICSU)",
    "Sewer Resilience Planning Supervisor",
    "Policy Writer",
    "PORTFOLIO MANAGER",
    "Director of the City Record",
    "Administrative Contract Specialist, M-I",
    "Systems Administrator - Computer Software",
    "2020-BWSO-001-Connections and Permitting Intern",
    "PROJECT SPECIALIST LEADER",
    "Managing Director, Green Infrastructure Planning and Partnerships",
    "Green Infrastructure Designer",
    "City Attendant",
    "VP - Operations",
    "Inspector",
    "Special Advisor to the Director, Neighborhood Development Areas (NDA) Initiative",
    "Product Manager",
    "Director, Borough Based Jail System",
    "Climber & Pruner",
    "DIRECTOR, OFFICE OF EQUITY, DIVERSITY & INCLUSION",
    "Director, Capital Budget",
    "NYC Census 2020 Lead Organizer",
    "2020-PS-001-Security Engineering/Engineering",
    "Emergency Management Planning Coordinator",
    "ANALYSTS  ADMINISTRATIVE AGENCIES",
    "Gardener 2",
    "Director of Materials Management",
    "Area Engineer",
    "Senior Investment Analyst â Alternative Credit (2 positions)",
    "Application Approver",
    "2020-EXEC/ENERGY-001-Energy Office Intern",
    "Custodian I",
    "SUPERVISING ATTORNEY",
    "Workforce Development Associate",
    "Director of Medical and Laboratory Facilities",
    "Chief, Infrastructure - Southeast Queens",
    "Telecom Associate - VOICE Technician",
    "Section Manager, Civil",
    "ENGINEERING TECHNICIAN 03",
    "Staten Island Office Deputy Director",
    "Deputy Agency Chief Contracting Officer (DAACO)",
    "Resident Engineer",
    "Field Technician - 7A Program",
    "Mechanical Inspector",
    "Supervising Inspector",
    "Analyst  FEMA Public Assistance Policy & Monitoring & Compliance / Monitoring & Compliance",
    "Chief Analyst/Project Manager",
    "Procurement Analyst",
    "Civil Engineering Intern",
    "AGENCY ATTORNEY 03",
    "IT Infrastructure Project Manager",
    "Deputy Commissioner of Policy",
    "DIRECTOR OF ENFORCEMENT",
    "Operations Analyst",
    "Director of Public Communications Structures",
    "IT Security Specialist - Security Audits (2 positions)",
    "Clerical Associate",
    "2020-BEDC-006-PMO Intern",
    "College Aide",
    "Evidence And Property Control Specialist I",
    "Building Information Modeling (BIM) Manager",
    "2020-BEDC-002-IHD Civil Intern",
    "Assistant Corporation Counsel, Brooklyn Tort Division",
    "Supervisor",
    "BUILDING INSPECTOR",
    "Landscape Architect II",
    "Analyst  DOITT, Finance, Law, Mayoralty & Elected Officials",
    "DATA ANALYST, BUSINESS OPERATIONS",
    "Unit Head  Reporting and Applications Development",
    "Structural Engineer",
    "2020-BEDC-001-BIM/CADD Intern",
    "Section Chief, Asset Management Maintenance Systems",
    "Carpenter",
    "Child Welfare Analyst",
    "Site Safety Plan Examination Intern",
    "Supervising Attorney-Employment",
    "Bureau Chief Contracting Officer",
    "2020-BEDC-004 WSCP Project Intern",
    "Network Administrator",
    "Special Counsel, Legal/Land Use",
    "Cyber Security Intern Security Science",
    "Legal Services Coordinator",
    "2020-BWS-010-Research Application/Innovation Intern",
    "Data Support Analyst",
    "Summer College Intern",
    "Program Data Analyst",
    "Environmental Educator",
    "Deputy Director of the Fire Alarm Inspection Unit",
    "Senior Advisor, Technology Strategy",
    "Buyer",
    "Directory Services Support",
    "2020-BWSO-003-Engineering & Regulatory Review Intern",
    "CoQI Facilitator",
    "2020-BWS-008-WRRO Intern (Tannersville)",
    "Business Analysis Specialist",
    "Junior Systems Administrator",
    "DIRECTOR, PROVIDER-CLIENT SERVICES & SPECIAL PROJECTS",
    "Analyst  Reporting and Applications Development",
    "Associate Public Health Sanitarian 2",
    "Director of Child Protective Audits",
    "Administrative Claim Examiner, No Fault Claims",
    "Rescue Tool & Equipment Coordinator",
    "Construction Project Manager Intern",
    "Instructor, Division of Environmental Health, Bureau of Environmental Administration (Health Academy)",
    "DIRECTOR, EXTERNAL AFFAIRS",
    "Transit Benefit Representative",
    "IT Support Technician",
    "IT Technical Support",
    "Forensic Pathology Coordinator",
    "Clinical Care Coordinator/Bureau of Mental Health",
    "2020-EHS-002-Reporting and Investigations Intern",
    "Deputy Engineer-In-Charge, Infrastructure",
    "Deputy Chief Of Outside Development",
    "2020-BWS-015-Hawthorne Field Intern",
    "2020-WT-007-Construction Management Intern",
    "Third Party Services Analyst",
    "Planning Associate",
    "Windows Administrator",
    "Analyst  Service Desk & Operations / Operations",
    "2020-BWSO-004-GIS Data Maintenance Intern",
    "2020-BWSO-005-Modeling Intern",
    "Minority Women Business Enterprise Program Analyst",
    "Structural Engineer (Retaining Walls and other Structural Assets)",
    "Assistant Engineering Audit Officer",
    "Analyst  DEPARTMENT OF ENVIRONMENTAL PROTECTION",
    "Assistant Commissioner of Community Partnerships and Finance",
    "Bridge Repairer & Riveter",
    "Salesforce Specialist",
    "Junior Public Health Nurse, Bureau of School Health",
    "FOOD INSECURITY ASSISTANCE TEAM SPECIALIST",
    "Senior Air Pollution Inspector",
    "2020-EHS-003-Program Development Intern",
    "Claims Manager - Law Claims (Non-Tort)",
    "Deputy Director for Engineering",
    "Community Engagement Planning Specialist",
    "2020-BWSO-002-Enforcement Intern",
    "Procurement Officer",
    "Outreach Coordinator",
    "Environmental Project Manager",
    "LEGAL SECRETARIAL ASSISTANT LEVEL 2",
    "Safety Education Outreach Coordinator",
    "Facilities Project Intern",
    "Diversity Specialist",
    "Civil Engineer 3",
    "ELECTRICIAN (AUTOMOBILE)",
    "AGENCY ATTORNEY III",
    "Office Manager",
    "In-House EHSO",
    "CLERICAL ASSOCIATE, L-IV",
    "Accountant",
    "Manager, Community Engagement",
    "Design Subject Matter Expert (SME)",
    "Senior Construction Contract Analyst",
    "Director of Finance",
    "Digital Media Specialist",
    "Asset Management Specialist",
    "In Rem (Foreclosure) Acquisitions Analyst",
    "Best Practices Coach",
    "Community Assistant",
    "Design Reviewer (Mechanical)",
    "Deputy Director of the Environmental Hazard Unit",
    "Dockless Bike Share Micromobility Planner",
    "Deputy General Counsel",
    "Associate Chemist",
    "ASSOCIATE STAFF ANALYST",
    "Director of Resiliency Planning",
    "Standards Specialist (Monitor)",
    "Printing Press Operator",
    "DIGITAL COMMUNICATIONS & GRAPHIC DESIGN SPECIALIST",
    "Environmental Health and Safety Officer",
    "Data Entry Clerk for Certificate of Fitness Unit",
    "Director of Facilities Maintenance",
    "Environmental Health & Safety Auditor",
    "Senior Design Reviewer (Mechanical)",
    "LABOR RELATIONS ATTORNEY",
    "Juniior Compliance Analyst",
    "Senior Estimator",
    "executive Vice President for Operations",
    "SENIOR ELECTRICAL ENGINEER",
    "Course Provider Program Intern",
    "Parks Enforcement Patrol",
    "2020-BWS-012-Water Supply Planning Intern",
    "Confidential Investigator 2",
    "Forenisc Mortuary Techncian II",
    "Director of Recruitment",
    "15BOROUGH COMMUNITY COORDINATOR",
    "Analyst - Business Practices Accountability Unit",
    "Electrical Engineer",
    "Assistant Commissioner",
    "Assistant Bureau Contract Officer",
    "2020-BWS-007-Water Treatment Ops QA Intern",
    "BUDGET ANALYST",
    "Public Health Advisor, Bureau of Sexually Transmitted Infections",
    "DIRECTOR-TECHNICAL SERVICES",
    "Alternative Assets Middle Office Analyst",
    "Section 3, MBWE and Procurement Officer",
    "Reliability Centered Maintenance Engineers",
    "Water Quality Scientist",
    "Engineer-In-Charge, Operational Information Systems",
    "Analyst  Transportation",
    "Director, Ombuds Office",
    "NYC Census 2020 - Brooklyn Borough Lead, NYC Census 2020 Field Campaign",
    "Housing Assistant",
    "Analyst  FEMA Public Assistance Fiscal & Reporting / Reporting",
    "HIV Program Associate, Bureau of STD Prevention and Control",
    "1022A (LEGAL SECRETARIAL ASST- L3)",
    "STAFF NURSE",
    "Director of STI Laboratory, Bureau of Sexually Transmitted Infections",
    "Topographical",
    "EXECUTIVE DIRECTOR, FINANCIAL AND REPORTING SYSTEMS",
    "Review Engineer",
    "Director of Technical Assistance",
    "Project Development Coordinator for Capital Contracts",
    "Civil Section Manager",
    "ESTIMATOR",
    "CERTIFIED IT ADMINISTRATOR (LAN/WAN), LEVEL 4",
    "Supervisor of Construction Services, Preservation Programs",
    "FOIL AND COMPLIANCE ASSOCIATE",
    "Clerical Associate, Bureau of Vital Statistics",
    "2020-RAM-001-Records and Archives Management",
    "Procurement Specialist",
    "Secretary 4",
    "2020-BWS-003-Water Ops Intern (Downsville)",
    "Accountable Manager, CSO LTCP Program",
    "Chief Communications Officer, Office of the Commissioner",
    "HRIS Data Analyst",
    "City Mortuary Technician",
    "Program Director, Health",
    "Area Engineer, BEDC Coordination and Project Development Section",
    "Examiner",
    "Assistant Corporation Counsel, Manhattan Tort Division",
    "Computer Associate (Technical Support) Level 3",
    "CASEWORKER",
    "Photographer",
    "Senior Compliance Manager",
    "Public Health Advisor, Bureau of Sexually Transmitted Diseases Prevention and Control",
    "RECRUITMENT PIPELINE AND PROFESSIONAL DEVELOPMENT PROGRAM MANAGER",
    "Deputy Director for Strategic Initiatives",
    "Deputy Unit Chief - Brooklyn Central Services",
    "Director of Talent Acquisition",
    "REGIONAL MANAGER, BROOKLYN REGION",
    "Associate Water Use Inspector I-Universal Metering",
    "Staff Auditor",
    "HQE Specialist",
    "Senior Project Lead",
    "Attorney Interne",
    "Deputy Chief Financial Officer",
    "Forensic Mortuary Technician I",
    "Agency Attorney, Level II",
    "CONTRACT ADMINISTRATOR",
    "Temporary Bronx Borough Planner",
    "SUPERVISOR OF STOCK WORKERS I",
    "Division Chief of Engineering",
    "Windows Technical Project Manager",
    "Parking Planner",
    "WORKFORCE1 OPERATIONS MANAGER",
    "ASSISTANT COMMISSIONER, RECRUITMENT AND ELIGIBILITY",
    "Admin Housing Dev Specialist for LHD EVP",
    "Human Resources Generalist",
    "NYC Census 2020 - Community Outreach Data Analyst",
    "Program Monitoring and Compliance Intern",
    "Engineer",
    "Compliance and Operations Specialist",
    "Civil Engineer 1",
    "Construction Safety and Quality Inspector",
    "Assistant Director for LHD Budget & Personnel",
    "Paralegal",
    "Voice Communication Specialist",
    "PEP Step Up Sergeant",
    "2020-BIT-001-Application Developer Intern",
    "Specialized Contract  Analyst  (2 positions)",
    "Executive Project Manager",
    "Media Service Tech",
    "Associate Water Use Inspector II",
    "Project Controls Manager",
    "Alternative Mechanical Systems Engineer",
    "Capital Projects and Budget Analyst",
    "Body Worn Camera Analyst",
    "Agency Attorney Interne, Office of General Counsel",
    "Vice President of Finance, Accounting and Investment Services",
    "Dynamic CRM Developer",
    "Program Executive",
    "SECTION SUPERVISOR",
    "Deputy Commissioner, Child and Family Well-Being",
    "Paralegal Aide",
    "CONSTRUCTION MANAGER",
    "Custodian Supervisor",
    "Timekeeping Specialist",
    "PARALEGAL",
    "MARINER",
    "Division Chief - Fiscal Services",
    "Procurement & Compliance Analyst",
    "Senior Systems Administrator",
    "Outcome Specialist, Contract Development & Validation",
    "Policy Analyst",
    "CCC Program Administrator",
    "Family and Community Engagement Liaison",
    "Director of Research and Analysis",
    "Exterminator",
    "Executive Assistant to the Director of Personnel & Payroll",
    "Human Capital Summer College Intern",
    "Criminalist III",
    "Contracts Coordinator for FHNR",
    "Associate Human Rights Specialist",
    "Regional Manager",
    "Analyst  Revenue Accountability",
    "EXECUTIVE DIRECTOR, IN-HOUSE DESIGN",
    "Network Engineer I LAN/WAN",
    "Policy Associate",
    "General Services Manager",
    "HR PAYROLL ANALYST",
    "COMPUTER ASSOC (SOFTWARE)",
    "Eligibility Coordinator / Adoption Liaison",
    "Program Director",
    "Paralegal Assistant Supervisor",
    "Associate Water Use Inspector II- Brooklyn",
    "PEP Step Up Captain",
    "JUNIOR SOFTWARE ENGINEER",
    "Junior Information Security Vulnerability Management Analyst",
    "STORAGE/BACKUP ADMINISTRATOR",
    "Clinical Director for the Couseling Services Unit",
    "Supply Clerk",
    "Software Support Analyst",
    "Senior Project Administrator",
    "Supervising Inspector of the Shelter Task Force",
    "Training Coordinator",
    "Court Liaison Officer",
    "CITY ASSESSOR",
    "Engineer In Charge of Pumping",
    "Resident Building Superintendent",
    "CPM Scheduler II",
    "Watershed Maintainer - 4 Positions",
    "ASSOCIATE ZONING SPECIALIST",
    "Assistant Commissioner of Quality Assurance for Juvenile Justice Programs",
    "First Associate Commissioner",
    "Inspector, Bureau of Environmental Disease and Injury Prevention",
    "GIS Technician",
    "Family Engagement Specialist",
    "Executive Director, Health Policy, Bureau of Primary Care Access & Planning",
    "Motor Vehicle Operator",
    "Special Projects Coordinator, Division of Neighborhood Preservation",
    "Executive Receptionist, Office of the Commissioner",
    "Unit Head  Program Analysis",
    "Assistant Procurement Liaison",
    "Senior Construction Manager",
    "Senior Estimator, General Construction",
    "Clerical Supervisor (JD)",
    "Technical Advisor for Reservoir Releases and Policy Development",
    "Risk Management Supervisor",
    "Trauma-Informed Early Care and Education Case Supervisor",
    "Zoning Deputy Director",
    "HIGH PRESSURE PLANT TENDER",
    "LINUX SYSTEMS ADMINISTRATOR",
    "Driver Services Agent",
    "BEHAVIORAL SPECIALIST",
    "Facilities Central Supervisor",
    "Application Support Analyst",
    "BUDGET MANAGER FOR PROGRAM OPERATIONS",
    "Director, Open Door Program, Division of New Construction Finance",
    "AUTO MECHANIC (DIESEL)",
    "311 Escalation Line Team Leader",
    "Child Welfare Trainer",
    "EXECUTIVE DIRECTOR, CONSTRUCTION INDUSTRY PARTNERSHIP",
    "Human Capital Associate",
    "Civil Engineer III",
    "Analyst  Criminal Justice",
    "Juvenile Justice Trainer",
    "Deputy Engineer-in-Charge, Southeast Queens",
    "Bluebelt Program Administrative Supervisor",
    "Investigative Auditor",
    "Superintendent of Water and Sewer Systems",
    "Public Health Assistant, Bureau of School Health",
    "Health Benefits Coordinator",
    "FDNY Supervisor of Telephone Adjudications",
    "Senior Court Representative, Litigation Division (2 positions)",
    "EVP for Real Estate",
    "Senior Financial Reporting Investment Analyst",
    "Associate Cloud Reliability Engineer",
    "Senior Correctional Institution Administrator, PREA",
    "AGENCY ATTORNEY INTERN",
    "Public Health Nurse (School Health)",
    "RFP COORDINATOR",
    "Constituent Services Liaison",
    "Placement Specialist",
    "DEPUTY PORTFOLIO MANAGER",
    "WORKFORCE QUALITY ASSURANCE MANAGER",
    "Agency Attorney I - Early Intervention Unit",
    "ADMINISTRATIVE ASSISTANT, LABOR SERVICES",
    "STAFF ANALYST II",
    "Civil/Structural Inspector",
    "Senior Network Design Engineer",
    "Assistant Commissioner of Neighborhood Development Stabilization",
    "Analyst  SANITATION",
    "Assistant Construction Inspector",
    "ADMINISTRATIVE PROCUREMENT ANALYST",
    "Leave & Workers' Compensation Coordinator",
    "METAL WORK MECHANIC",
    "Associate Project Manager III",
    "Supervisor of Traffic Device Maintainers",
    "Supervising Budget Analyst Early Care &Education",
    "Environmental Health & Safety Officer",
    "Executive Director of Crossover Youth Practice",
    "Labor Economist",
    "Agency Attorney, Level III",
    "Caretaker X",
    "Senior Estimator (General Construction)",
    "2020-WT-005-Process and Research Development",
    "OPERATION ASSISTANT",
    "Agency Attorney 1",
    "Traffic Control Inspector",
    "Senior Urban Designer",
    "Deputy Chief of Facilities - Services",
    "Quality Assurance / Technical Litigation Support Analyst",
    "BUDGET SUPPORT ASSISTANT",
    "ADMINISTRATIVE MANAGEMENT AUDITOR",
    "Analyst  Housing",
    "Crime Strategy Analyst",
    "Accountant - Contract Payment",
    "Case Manager",
    "TECH SUPPORT/SCHEDULER",
    "Architect L2",
    "FIA Worker-Facing Information Technology Project Specialist",
    "Deputy Director of Continued Occupancy",
    "Quality Assurance Specialist L1",
    "Assistant Commissioner of Compliance and Legal Oversight",
    "Design Reviewer (Electrical)",
    "Analyst  Correction",
    "Investigator Trainee",
    "Agency Attorney 03",
    "Clerical Associate (Part-Time)",
    "Analyst  ECONOMIC DEVELOPMENT",
    "Associate Program Manager",
    "Neighborhood Preparedness Training Coordinator",
    "Housing Manager",
    "Chief of Staff for Deputy Commissioner",
    "ASSISTANT ELECTRICAL ENGINEER",
    "Associate General Counsel / Law Department / General Litigation",
    "Help Desk Technician",
    "Business and Data Analyst Manager",
    "Supervising Housing Inspector for the Division of Code Enforcement",
    "Senior Application Developer",
    "Agency Security Director",
    "Deputy Chief of Quality Assurance",
    "Junior Environmental Planner",
    "SENIOR PREVAILING WAGE INVESTIGATOR",
    "Project Manager, Administration",
    "Agency Attorney",
    "Senior SQL /Oracle Database Administrator",
    "Engineering",
    "EH&S Program Manager",
    "Platform Engineer",
    "PEOPLESOFT DEVELOPER",
    "PSYCHOLOGIST",
    "Senior Strategic Planning Analyst",
    "Associate Commissioner",
    "Base-Business Licensing Specialist",
    "Physician Support Assistant, Bureau of School Health",
    "Emergency Planning Coordinator, Environmental Health Administration",
    "Auto Mechanic",
    "Accountable Project Manager,  Stormwater MS4 Evaluation and Alternatives",
    "Clerical Associate Part Time",
    "ASSISTANT URBAN DESIGNER",
    "Fire Protection Engineer",
    "Personnel Associate",
    "High Pressure Plant Tender",
    "Program Associate, ACE Program",
    "Assistant Commissioner, Internal Communications",
    "DISASTER RECOVERY ANALYST.",
    "Desktop Support Engineer",
    "Communications Specialist",
    "Retrofit Accelerator Outreach and Marketing Coordinator",
    "Health Benefits Program Coordinator",
    "Office Manager (Brooklyn)",
    "CASE MANAGEMENT NURSE",
    "Assistant Records Officer",
    "Watershed Maintainer - 2 positions",
    "Graphic Designer",
    "Appeals Assistant District Attorney",
    "Vice-President for Support Services",
    "Hardware Engineer (MAINFRAME)",
    "Code and Zoning Specialist",
    "Assistant Commissioner, Tax Policy",
    "Service Desk Manager",
    "Project Manager, Strategic Planning",
    "Central Office Supervising Nurse, Bureau of School Health/SH Nursing Services & Prof Dev",
    "Account Manager",
    "Strategic Initiative Specialist",
    "Structural Engineering Intern (Retaining Walls and other Structural Assets)",
    "Environmental Planner",
    "Payroll Analyst",
    "2020-LRD-001 Labor Relations Intern",
    "Law Student",
    "FIA AUDIT COORDINATOR",
    "Electrical Engineer 3",
    "Associate Director / Deputy Director  Expense & Capital Budget",
    "Deputy Director of Engineering Services",
    "NYC Census 2020 Cultural Liaison",
    "Driver Assistance Center Intake Specialist",
    "Computer Aide Level 2",
    "Information Security Engineer",
    "Team Leader",
    "Application Support Technician",
    "Certified IT Administrator (LAN/WAN) Level II",
    "AOT Supervisor, Mental Health",
    "Deputy Director of Support and Special Services, M-II",
    "Plumber",
    "Quality Improvement Specialist",
    "Assistant Inspector General",
    "ANALYST  PARKS",
    "Trees & Sidewalks Program Coordinator",
    "Chief, Personnel Unit",
    "Senior Policy Advisor, Energy Finance and Affordability",
    "Human Resources Administrative Assistant",
    "Grants Coordinator",
    "Director, CCEWIN Operations",
    "Asthma Program Social Worker, Bureau of School Health/SH Medical",
    "Chief of Staff",
    "Mediation Coordinator",
    "Engineering Auditor",
    "Designated Felony Assistant Corporation Counsel",
    "CONTRACT MANAGER, PROGRAM FINANCE & OPERATIONS",
    "City Planner",
    "Desktop Support Technician",
    "Associate Project Manager I",
    "Junior Business Analyst",
    "Senior Landscape Program Manager",
    "Management Auditor, Level II",
    "Programmer, Bureau of Environmental Disease and Injury Prevention",
    "PREA Project Director",
    "Director of DCAS Trades Shop",
    "Assistant District Attorney",
    "COLLEGE AIDE - CLERICAL",
    "Technical Assistant",
    "Deputy Administrator for Washington Square Park",
    "Technical Policy Researcher",
    "ASSISTANT ELECTRICAL INSPECTOR",
    "Assistant Electrical Engineer",
    "Medical Examiner Assistant",
    "Social Worker, Bureau of Sexually Transmitted Infections",
    "Senior Forester for Tree Procurement",
    "Office Assistant (Part-Time)",
    "Administrative Assistant to the Deputy Commissioner for Uniformed Services Bureau",
    "PARALEGAL AIDE - LEVEL 2",
    "FMS ARCHITECT/DEVELOPER",
    "CATCH School Nurse, Bureau of School Health",
    "Seasonal City Park Worker",
    "Staten Island Construction & Performance Monitor",
    "Conference Facilitator",
    "Dockbuilder",
    "Senior Developer, MO for Economic Opportunity",
    "WEB / Application Supervisor",
    "Bike Share Program Planner",
    "Taxpayer Case Advocate",
    "OFFICE ENGINEER",
    "Deputy Director of Professional Development Program",
    "Analysis and Reporting Administrator",
    "2020-BWS-011-WTO Engineering & Contract Services Intern",
    "Stock Worker",
    "ADMINISTRATIVE HOUSING SUPERINTENDENT",
    "ASBESTOS HAZARD INVESTIGATOR",
    "Agency Attorney Level III",
    "Technical Response Technician",
    "2020-BWS-004-Kingston Field Intern",
    "Network Specialist",
    "BTS Senior Application Database Administrator",
    "Deputy Chief, Industrial Pollution Prevention Unit",
    "Associate Laboratory Microbiologist I",
    "Queens Business Center Manager",
    "Communications Director",
    "TIMEKEEPER",
    "MANAGEMENT AUDITOR",
    "Assistant District Manager",
    "M/WBE Outreach and Compliance",
    "2020-BEDC-005 WWCP Project Intern",
    "Secretary to Division Chief",
    "Junior Public Health Nurse - Bureau of School Health/SH Nursing Services & Prof Dev",
    "Records Associate",
    "Civilianization Manager",
    "Database Administrator",
    "Deputy Director of Initial Subsidies",
    "Information Storage Engineer",
    "CITY RESEARCH SCIENTIST II",
    "Senior Compliance Advisor",
    "Supervisor of Construction Services, New Construction Programs",
    "ASSISTANT COMMISSIONER FOR FACILITIES PLANNING AND ENGINEERING",
    "Engineer In Charge, Linear Capital Program Management",
    "Project Manager (Modeling)",
    "Director - Technical Services",
    "E-Discovery Specialist",
    "Cyber Budget/Procurement Liaison",
    "Stationary Engineer, Bureau of Operations",
    "Claims Analyst",
    "Project Manager  Value Engineer",
    "Staff Development Coordinator",
    "Associate Water Use inspector",
    "PARALEGAL - EARLY INTERVENTION UNIT",
    "Deputy Chief of Forestry & Horticulture",
    "OUTREACH AND RETENTION SPECIALIST",
    "Elevator Oversight Team Specialist",
    "Post Hearings Clerk",
    "Deputy Chief of Payments and Procurement",
    "NYC Census 2020 Campaign Manager",
    "Associate Email Security Engineer",
    "Website Publisher",
    "Senior Compliance Investigator",
    "Chief Data Science Officer",
    "Associate Borough Planner",
    "Relationship Manager",
    "Director of Security",
    "Revenue Support Specialist",
    "Learning & Training Development Specialist",
    "Public Health Assistant, Bureau of Sexually Transmitted Infections",
    "Assistant Mechanical Engineer (Construction Classification)",
    "QUALITY CONTROL CASE REVIEW SUPERVISOR",
    "COMPUTER SPECIALIST (OPERATIONS)",
    "Program & Policy Specialist for Racial and Restorative Justice",
    "Payroll Clerk",
    "Clerical Support",
    "PEP Sergeant",
    "Division Chief Asset and Construction Management",
    "Chief of Review and Permitting",
    "PRINCIPAL ADMINISTRATIVE ASSOCIATE",
    "Union Services Associate",
    "Investigator Level 1",
    "SharePoint Developer",
    "AOTPS Budget Manager",
    "Check Replacement Analyst",
    "FIRES IT Supervisor",
    "Cornerstone Program Director",
    "Supervisor II (SW)",
    "Deputy Chief of Facilities, Supplies & Technical Support",
    "Community Engagement Specialist",
    "Brooklyn Forestry Coordinator",
    "Facility Manager",
    "Staff Analyst Level 2",
    "Applications Supervisor",
    "Assisstant Supt",
    "Administrative Manager (Non-Mgrl)",
    "Stock Worker Level II",
    "RESIDENT BUILDING SUPERINTENDENT",
    "Union Deduction Analyst",
    "Supervisor of Stock Worker",
    "Plant Chief",
    "2020-BLA-002 Graduate Legal Intern",
    "Testifier / Searcher",
    "Geotab Fleet Liaison",
    "Equipment Training Coordinator",
    "Deputy Chief of Information Technology",
    "Information Technology Manager",
    "Maintenance Worker",
    "Staff Analyst, Level II",
    "Director of Legislative Affairs",
    "Accounting Analyst",
    "Management Auditor, Level I"
]

In [7]:
# Load infersent model 
model_version = 2
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048, 'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}

model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

# generate infersent sentence embeddings
model.build_vocab(sentences, tokenize=True)
embs = model.encode(sentences, tokenize=True)

Found 1071(/1199) words with w2v vectors
Vocab size : 1071


In [0]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(embs)
y_km = kmeans.fit_predict(embs)

In [14]:
for i in range(6): print(f'cluster {i}', len([x for idx, x in enumerate(sentences) if y_km[idx] == i]))
titles_dict = [{'title': s, 'category': y_km[i]} for i, s in enumerate(sentences)] 

cluster 0 151
cluster 1 64
cluster 2 277
cluster 3 179
cluster 4 273
cluster 5 295


In [15]:
print(titles_dict)

[{'title': 'Executive Director for Tax', 'category': 4}, {'title': 'ASSOCIATE PUBLIC HEALTH SANITARIAN II', 'category': 3}, {'title': 'Investigator', 'category': 1}, {'title': 'MAINFRAME STORAGE MANAGER', 'category': 3}, {'title': 'Civil Engineer 2', 'category': 4}, {'title': '2020-WT-008-Asset Management Intern', 'category': 2}, {'title': 'VP- Operations', 'category': 5}, {'title': 'IT Project Manager/Business Analyst', 'category': 2}, {'title': 'MAINTENANCE WORKER', 'category': 3}, {'title': 'Project Advocate', 'category': 2}, {'title': 'Labor Compliance Analyst', 'category': 2}, {'title': 'Forestry Program Manager', 'category': 2}, {'title': 'JAVA DEVELOPER', 'category': 3}, {'title': 'DEPUTY DIRECTOR -  Strategic Initiatives &Analysis', 'category': 4}, {'title': 'City Seasonal Aide', 'category': 2}, {'title': 'EXECUTIVE DIRECTOR, REGULATORY REFORM', 'category': 3}, {'title': 'Deputy Director Conversions/HQE', 'category': 5}, {'title': 'Administrative Assistant', 'category': 5}, {'t